In [30]:
import numpy as np
import pandas as pd


### Exercise 2

### 2.1

In [31]:
def sup(D, X, Y=None):
    supp = 0
    if Y is None:
        for transaction in D:
            if all(item in transaction for item in X):
                supp += 1
    else:
        for transaction in D:
            if all(item in transaction for item in X) and all(item in transaction for item in Y):
                supp += 1
    return supp

def conf(D, X, Y):
    return sup(D, X, Y) / sup(D, X)

def lift(D, X, Y=None):
    if Y is None:
        Y = X.copy()
    supports = [sum(all(item in transaction for item in items) for transaction in D) for items in (X, Y, X + Y)]
    if 0 in (r_support := [sup / len(D) for sup in supports]):
        return None
    return r_support[2] / (r_support[0] * r_support[1])

def leverage(D, X, Y):
    supports = [sum(all(item in transaction for item in items) for transaction in D) for items in (X, Y, X + Y)]
    final_leverage = supports[2] / len(D) - (supports[0] / len(D)) * (supports[1] / len(D))
    return final_leverage

def jaccard(D, X, Y):
    return sup(D, X, Y) / (sup(D, X) + sup(D, Y) - sup(D, X, Y))

def conviction(D, X, Y):
    support_y = sum(all(item in transaction for item in Y) for transaction in D) / len(D)
    confidence_xy = conf(D, X, Y)
    if confidence_xy >= 1:
        return None
    return (1 - support_y) / (1 - confidence_xy)

def oddsRatio(D, X, Y):
    supports = {(False, False): 0, (False, True): 0, (True, False): 0, (True, True): 0}
    for transaction in D:
        supports[(all(item in transaction for item in X), all(item in transaction for item in Y))] += 1
    final_odds_ratio = ((supports[(True, True)] + 1) * (supports[(False, False)] + 1)) / ((supports[(False, True)] + 1) * (supports[(True, False)] + 1))
    return final_odds_ratio

def imp(D, X, Y):
    if len(X) < 2:
        return 0
    conf_xy = conf(D, X, Y)
    w_conf = [conf(D, W, Y) for W in X]
    return conf_xy - max(w_conf) if w_conf else 0

def getRuleMetric(D,X,Y,metric):
    if metric == "sup":
        return sup(D,X,Y)
    elif metric == "conf":
        return conf(D,X,Y)
    elif metric == "lift":
        return lift(D,X,Y)
    elif metric == "leverage":
        return leverage(D,X,Y)
    elif metric == "jaccard":
        return jaccard(D,X,Y)
    elif metric == "conviction":
        return conviction(D,X,Y)
    elif metric == "oddsratio":
        return oddsRatio(D,X,Y)
    elif metric == "imp":
        return imp(D,X,Y)
    else:
        return "metrica invalida"
    


### 2.2

In [32]:
def filterProductiveRules(D, R):
    productive_rules = []
    for rule in R:
        X, Y, sup_XY, conf_XY = rule
        if conf_XY > 0:
            productive_rules.append(rule)
    return productive_rules

### 2.3